In [1]:
import pandas as pd
import numpy as np

In [2]:
dfcatalogo = pd.read_csv(r'C:\Users\oomph\Documents\Henry Data Science\DTS09_Proyecto_Individual\df_catalogo_v2.csv')


funcion get max duration

In [4]:
def get_max_duration(year: int, platform: str, duration_type: str) -> dict:
    # Filtrar el dataframe por tipo de contenido, año y plataforma
    filtered_df = dfcatalogo[(dfcatalogo['type'] == 'movie') & 
                            (dfcatalogo['release_year'] == year) & 
                            (dfcatalogo['plataforma'] == platform)]
    
    # Obtener la película con la duración máxima en el tipo especificado
    max_duration = filtered_df[filtered_df['duration_type'] == duration_type]['duration_int'].max()
    max_duration_movie = filtered_df[(filtered_df['duration_int'] == max_duration) &
                                     (filtered_df['duration_type'] == duration_type)]['title'].values
    
    # Verificar que el array de resultados no está vacío antes de intentar acceder a su primer elemento
    if len(max_duration_movie) > 0:
        return {'pelicula': max_duration_movie[0]}
    else:
        return {'pelicula': ''}

In [5]:
# Ejemplo de llamado a la función get_max_duration
year = 2016
platform = 'amazon'
duration_type = 'min'

max_duration_movie = get_max_duration(year, platform, duration_type)
print(f'La película con duración máxima en el año {year} en la plataforma {platform} en {duration_type} es: {max_duration_movie}')

La película con duración máxima en el año 2016 en la plataforma amazon en min es: {'pelicula': 'gentle evening rain'}


funcion get score count

In [6]:
def get_score_count(df, plataforma, scored, anio):
    # Seleccionar solo las películas de la plataforma y el año especificado
    peliculas = df[(df['plataforma'] == plataforma) &
                   (df['type'] == 'movie') &
                   (df['release_year'] == anio)]
    # Filtrar solo las películas con un puntaje mayor al especificado
    peliculas_con_puntaje = peliculas[peliculas['avg_rating'] > scored]
    # Obtener la cantidad de películas que cumplen los criterios
    cantidad = len(peliculas_con_puntaje)
    return {
        'plataforma': plataforma,
        'cantidad': cantidad,
        'anio': anio,
        'score': scored
    }

In [7]:
get_score_count(dfcatalogo, 'disney+', 1.45, 2014)

{'plataforma': 'disney+', 'cantidad': 34, 'anio': 2014, 'score': 1.45}

funcion get count platform

In [8]:
def get_count_platform(df, plataforma):
    # Seleccionar solo las películas de la plataforma especificada
    peliculas = df[(df['plataforma'] == plataforma) &
                   (df['type'] == 'movie')]
    # Obtener la cantidad de películas que cumplen los criterios
    cantidad = len(peliculas)
    return {
        'plataforma': plataforma,
        'peliculas': cantidad
    }

In [9]:
get_count_platform(dfcatalogo, 'hulu')

{'plataforma': 'hulu', 'peliculas': 1484}

funcion get actor

In [10]:
def get_actor(platform:str, year:int ):
    df_filtered = dfcatalogo[(dfcatalogo['plataforma'] == platform) & (dfcatalogo['release_year'] == year)]
    actor_count = {}
    for actors in df_filtered['cast']:
        if isinstance(actors, str):
            for actor in actors.split(','):
                actor_count[actor.strip()] = actor_count.get(actor.strip(), 0) + 1
    if len(actor_count) == 0:
        return "No se encontraron actores"
    else:
        return 'Actor que más se repite según plataforma y año:  ' + max(actor_count, key=actor_count.get)

In [11]:
respuesta = get_actor('amazon', 2010)
print(respuesta)

Actor que más se repite según plataforma y año:  cassandra peterson


funcion prod per county

In [14]:
def prod_per_county(tipo: str, pais: str, anio: int):
    # Filtramos el DataFrame según el tipo de contenido, el país y el año
    df_filtered = dfcatalogo[(dfcatalogo['type'] == tipo) & (dfcatalogo['country'] == pais) & (dfcatalogo['release_year'] == anio)]

    # Contamos el número total de elementos del DataFrame filtrado
    total_peliculas = df_filtered.shape[0]

    return {'pais': pais, 
            'anio': anio, 
            'total de contenido': total_peliculas,
            'tipo_contenido': tipo}

In [15]:
respuesta = prod_per_county('tv show', 'france', 2020,)
print(respuesta)

{'pais': 'france', 'anio': 2020, 'total de contenido': 8, 'tipo_contenido': 'tv show'}


funcion get contents

In [16]:
def get_contents(rating: str):
    # Utilizamos value_counts para contar el número de contenidos según el rating de audiencia
    contenido = dfcatalogo['rating'].value_counts().get(rating, 0)

    return {'rating': rating, 'contenido': contenido}

In [17]:
respuesta = get_contents('r')
print(respuesta)

{'rating': 'r', 'contenido': 2154}


In [18]:
dfcatalogo.sample(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,movieId,duration_int,duration_type,avg_rating
5563,s5564,movie,prisoner number a26188: surviving auschwitz,lisa bryer,NaN,NaN,NaN,2013,13+,44 min,"documentary, special interest",the german invasion of poland in 1939 marked t...,amazon,as5564,44.0,min,3.563181
9172,s9173,movie,je suis a stand-up: eddie izzard abroad,paul tickell,eddie izzard,NaN,NaN,1996,18+,48 min,"arts, entertainment, and culture, comedy, spec...",bbc2 documentary of eddie izzard's first foray...,amazon,as9173,48.0,min,3.531659
13479,s2362,movie,johnny reno,NaN,NaN,united states,2018-11-01,1966,tv-g,83 min,"action, adventure",one man stands alone against the fury of mob j...,hulu,hs2362,83.0,min,3.571577
17533,s3343,tv show,queer eye: we're in japan!,NaN,"bobby berk, karamo brown, tan france, antoni p...","united states, japan",2019-11-01,2019,tv-14,1 season,"international tv shows, reality tv",the fab five touch down in tokyo to spread the...,netflix,ns3343,1.0,season,3.580550
10281,s614,movie,viking warrior women,stuart strickson,ella al-shamahi,united kingdom,2020-02-28,2019,tv-pg,44 min,"documentary, historical",new discoveries challenge everything we though...,disney+,ds614,44.0,min,3.519651
19243,s5053,movie,liberated: the new sexual revolution,NaN,NaN,"united states, mexico",2018-02-01,2017,tv-ma,86 min,documentaries,filmmakers explore how pop culture shapes a ne...,netflix,ns5053,86.0,min,3.542074
3081,s3082,tv show,bhoomi hindi,NaN,"naziya sheik, arun babu",NaN,NaN,2021,18+,1 season,drama,bhoomi got raped by a well know business man s...,amazon,as3082,1.0,season,3.582292
8956,s8957,movie,elvira's movie macabre: scared to death,"d.j. viola, christy cabanne",cassandra peterson,NaN,NaN,2010,16+,90 min,"comedy, horror, suspense","a revamping, if you will, of the 1980's classi...",amazon,as8957,90.0,min,3.565760
1271,s1272,movie,prayer never fails,wes miller,"corbin bernsen, lorenzo lamas, eric roberts, n...",NaN,NaN,2017,7+,98 min,"drama, faith and spirituality",when a high school basketball coach is fired f...,amazon,as1272,98.0,min,3.459432
19692,s5502,movie,he even has your eyes,lucien jean-baptiste,"lucien jean-baptiste, aissa maiga, zabou breit...","france, belgium",2017-05-01,2016,tv-ma,95 min,"comedies, international movies",when french-african couple paul and sali disco...,netflix,ns5502,95.0,min,3.513752
